In [87]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [88]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [94]:
data_dir = '/content/drive/MyDrive/Colab Notebooks/ibb_ass3/torch2'
model_name = "resnet"
num_classes = 100
batch_size = 10
num_epochs = 20
feature_extract = True

In [95]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [96]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [97]:
model = models.resnet50(pretrained=True)
feats = model.fc.in_features
model.fc = nn.Linear(feats, num_classes)


In [98]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


Initializing Datasets and Dataloaders...


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [99]:
model = model.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized    
#learning_rate = 0.001
#optimizer_ft = optim.Adam(params_to_update, lr=learning_rate)
optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Params to learn:
	 conv1.weight
	 bn1.weight
	 bn1.bias
	 layer1.0.conv1.weight
	 layer1.0.bn1.weight
	 layer1.0.bn1.bias
	 layer1.0.conv2.weight
	 layer1.0.bn2.weight
	 layer1.0.bn2.bias
	 layer1.0.conv3.weight
	 layer1.0.bn3.weight
	 layer1.0.bn3.bias
	 layer1.0.downsample.0.weight
	 layer1.0.downsample.1.weight
	 layer1.0.downsample.1.bias
	 layer1.1.conv1.weight
	 layer1.1.bn1.weight
	 layer1.1.bn1.bias
	 layer1.1.conv2.weight
	 layer1.1.bn2.weight
	 layer1.1.bn2.bias
	 layer1.1.conv3.weight
	 layer1.1.bn3.weight
	 layer1.1.bn3.bias
	 layer1.2.conv1.weight
	 layer1.2.bn1.weight
	 layer1.2.bn1.bias
	 layer1.2.conv2.weight
	 layer1.2.bn2.weight
	 layer1.2.bn2.bias
	 layer1.2.conv3.weight
	 layer1.2.bn3.weight
	 layer1.2.bn3.bias
	 layer2.0.conv1.weight
	 layer2.0.bn1.weight
	 layer2.0.bn1.bias
	 layer2.0.conv2.weight
	 layer2.0.bn2.weight
	 layer2.0.bn2.bias
	 layer2.0.conv3.weight
	 layer2.0.bn3.weight
	 layer2.0.bn3.bias
	 layer2.0.downsample.0.weight
	 layer2.0.downsample.1.weight

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)

In [ ]:
criterion = nn.CrossEntropyLoss()

# Train and evaluate
model, hist = train_model(model, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="inception"))

Epoch 0/14
----------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


train Loss: 4.7584 Acc: 0.0107
val Loss: 4.5970 Acc: 0.0049

Epoch 1/14
----------
train Loss: 4.4688 Acc: 0.0410
val Loss: 4.5212 Acc: 0.0243

Epoch 2/14
----------
train Loss: 4.0952 Acc: 0.0891
val Loss: 4.4119 Acc: 0.0291

Epoch 3/14
----------
train Loss: 3.6374 Acc: 0.2210
val Loss: 4.3175 Acc: 0.0388

Epoch 4/14
----------
train Loss: 3.1561 Acc: 0.3458
val Loss: 4.1627 Acc: 0.0874

Epoch 5/14
----------
train Loss: 2.6398 Acc: 0.5633
val Loss: 4.0258 Acc: 0.1019

Epoch 6/14
----------
train Loss: 2.0905 Acc: 0.7487
val Loss: 3.9195 Acc: 0.1117

Epoch 7/14
----------
train Loss: 1.6041 Acc: 0.8663
val Loss: 3.8519 Acc: 0.1456

Epoch 8/14
----------
train Loss: 1.2043 Acc: 0.9234
val Loss: 3.7971 Acc: 0.1942

Epoch 9/14
----------
train Loss: 0.8062 Acc: 0.9768
val Loss: 3.7491 Acc: 0.1796

Epoch 10/14
----------
train Loss: 0.5380 Acc: 0.9893
val Loss: 3.7104 Acc: 0.1990

Epoch 11/14
----------
train Loss: 0.3674 Acc: 0.9964
val Loss: 3.6611 Acc: 0.1845

Epoch 12/14
----------
t

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/ibb_ass3/modelresnet50scg-perfectlyannot.pth')

In [ ]:
import joblib as joblib


In [ ]:
joblib.dump([hist], '/content/drive/MyDrive/Colab Notebooks/ibb_ass3/histogram.joblib')

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
model.eval()

In [ ]:
from PIL import Image, ImageDraw
import torchvision.transforms
from torch.autograd import Variable


In [ ]:
def predict_image(path):
    #print("Prediction in progress")
    image = Image.open(path)

    transformation = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    img_tensor = transformation(image).float()
    img_tensor = img_tensor.unsqueeze_(0)

    #if torch.cuda.is_available():
    #    img_tensor.cuda()

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    img_tensor = img_tensor.to(device)
    
    input = Variable(img_tensor)
    output = model(input)
    index = output.cpu().data.numpy().argmax() + 1

    return index



In [ ]:
import numpy as np
import os
import csv
from keras.preprocessing import image
from openpyxl import load_workbook

wb = load_workbook('/content/drive/MyDrive/Colab Notebooks/ibb_ass3/recognition.xlsx')
dict1 = {}

for row in wb.worksheets[0].iter_rows():
   dict1[(row[0].value)] = row[1].value

directory = os.fsencode('/content/drive/MyDrive/Colab Notebooks/ibb_ass3/cropped2/')
count = 0;
for file in os.listdir(directory):
     #print('####'*10)
     filename = os.fsdecode(file)
     ImagePath = '/content/drive/MyDrive/Colab Notebooks/ibb_ass3/cropped2/' + filename;
     #print(ImagePath)
     prediction = predict_image(ImagePath)

     #print('Prediction is: ', prediction)
     #print('It should be: ', dict1["./test/" + filename])
     if (int(prediction) == int(dict1["./test/" + filename])):
       #print("SUCCESS");
       count = count + 1;
       #print(count);
     #with open('results.csv', mode='a') as employee_file:
      #employee_writer = csv.writer(employee_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
      #employee_writer.writerow([ImagePath, ResultMap[np.argmax(result)], sorted(zip(result[0], list(training_set.class_indices.keys())), reverse=True)[:3]])

print(count)